In [29]:
from data_handler import return_create_statement_from_dataframe
import pandas as pd
from db_handler import create_connection,execute_query, close_connection
from data_handler import insert_statement_from_dataframe
db_session = create_connection()

In [30]:

stg_users_dataframe = pd.DataFrame({
    "ID": [1, 2, 3],
    "Age": [25, 30, 35],
    "Gender": ['Male', 'Female', 'Male'],
    "fav_music_genre": ['Pop', 'Rock', 'Hip-Hop'],
    "music_time_slot": ['Morning', 'Evening', 'Afternoon'],
    "music_lis_frequency": ['Daily', 'Weekly', 'Monthly'],
    "music_satisfaction": ['High', 'Medium', 'Low']
})

stg_songs_dataframe = pd.DataFrame({
    "track_id": [1, 2, 3],
    "track_name": ["Song1", "Song2", "Song3"],
    "track_artist": ["Artist1", "Artist2", "Artist3"],
    "track_popularity": [80, 90, 70],
    "track_album_id": [101, 102, 103],
    "track_album_name": ["Album1", "Album2", "Album3"],
    "track_album_release_date": ["2022-01-01", "2022-02-01", "2022-03-01"],
    "playlist_name": ["Playlist1", "Playlist2", "Playlist3"],
    "playlist_id": [201, 202, 203],
    "playlist_genre---": ["Genre1", "Genre2", "Genre3"],
    "energy": [0.8, 0.7, 0.6],
    "key": [1, 2, 3],
    "loudness": [-5.0, -6.0, -7.0],
    "mode": [0, 1, 0],
    "speechiness": [0.1, 0.2, 0.3],
    "valence": [0.7, 0.6, 0.5],
    "tempo": [120, 130, 140],
    "duration_ms": [240000, 250000, 260000]
})




In [31]:

stg_users_table_name = "stg_users_data"
stg_users_schema_name = "public"  
stg_users_create_statement = return_create_statement_from_dataframe(
    stg_users_dataframe, stg_users_schema_name, stg_users_table_name)


stg_songs_table_name = "stg_songs_data"
stg_songs_schema_name = "public"  
stg_songs_create_statement = return_create_statement_from_dataframe(
    stg_songs_dataframe, stg_songs_schema_name, stg_songs_table_name)

execute_query(conn= db_session, query= stg_users_create_statement)

execute_query(conn= db_session, query= stg_songs_create_statement)

In [32]:
query = insert_statement_from_dataframe(stg_songs_dataframe,stg_songs_schema_name , stg_songs_table_name)
execute_query(db_session,query)
print(query)


INSERT INTO public.stg_songs_data (track_id, track_name, track_artist, track_popularity, track_album_id, track_album_name, track_album_release_date, playlist_name, playlist_id, playlist_genre---, energy, key, loudness, mode, speechiness, valence, tempo, duration_ms) VALUES (1, ''Song1'', ''Artist1'', 80, 101, ''Album1'', ''2022-01-01'', ''Playlist1'', 201, ''Genre1'', 0.8, 1, -5.0, 0, 0.1, 0.7, 120, 240000, 2, ''Song2'', ''Artist2'', 90, 102, ''Album2'', ''2022-02-01'', ''Playlist2'', 202, ''Genre2'', 0.7, 2, -6.0, 1, 0.2, 0.6, 130, 250000, 3, ''Song3'', ''Artist3'', 70, 103, ''Album3'', ''2022-03-01'', ''Playlist3'', 203, ''Genre3'', 0.6, 3, -7.0, 0, 0.3, 0.5, 140, 260000);
